In [ ]:
#!pip install chromadb sentence-transformers

In [1057]:
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [1058]:
article_info = pd.read_csv('data/article_info.csv')
view_log = pd.read_csv('data/view_log.csv')

In [1060]:
from  langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings



In [1061]:
def embed_file(CACHE_DIR, model_name = 'BAAI/bge-m3'):
    
    
    model_kwargs = {
        # "device": "mps"
        "device": "cpu"
    }
    encode_kwargs = {"normalize_embeddings": True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        cache_folder = CACHE_DIR
    )
    
    return embeddings

In [1062]:
embedding = embed_file('pretrained/')

In [1063]:
docs = [
    Document(page_content = row['Title'], metadata = {'id' : row['articleID']}) for idx, row in article_info.iterrows()
]

In [1064]:
vectorstore = Chroma.from_documents(docs, embedding) # 시간걸림


In [1067]:
from tqdm import tqdm

reco_dics = {id : {'docs' : [], 'scores' : []} for id in view_log['articleID'].unique()}

for article_id in tqdm(view_log['articleID'].unique()):
    title = article_info[article_info['articleID'] == article_id]['Title'].iloc[0]
    for relevance_data in vectorstore.similarity_search_with_relevance_scores(title, k =5):
        doc, score = relevance_data
        if  article_id != doc.metadata['id']:
            
            reco_dics[article_id]['docs'].append(doc.metadata['id'])
            reco_dics[article_id]['scores'].append(score)


100%|██████████| 2879/2879 [08:12<00:00,  5.85it/s]


In [ ]:
person_reco_dics = {id : [] for id in view_log['userID'].unique()}

for user_id in tqdm(view_log['userID'].unique()):
    
    view_articles = view_log[view_log['userID'] == user_id]['articleID'].values
    
    for article_id in view_articles:
        person_reco_dics[user_id] += reco_dics[article_id]
    
